In [1]:
import pandas as pd
import os

In [3]:
df = pd.read_csv('../data/winequality-red.csv')

In [8]:
# function to split and save data, takes 10 rows and indexes the filenames
def split_and_save_data(df):
    while not df.empty:
        chunk = df.sample(n=min(10, len(df)))
        filename = f'data_chunk_{len(os.listdir("../raw_data")) + 1}.csv'
        
        chunk.to_csv(f'../raw_data/{filename}', index=False)
        df = df.drop(chunk.index)
        
        if df.empty:
            print("DataFrame is empty. Exiting...")
            break

In [9]:
split_and_save_data(df)

DataFrame is empty. Exiting...
